In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
import sys

SOURCE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__name__)))
sys.path.insert(0, SOURCE_DIR)

In [3]:
import tensorflow as tf
import malaya_speech
import malaya_speech.train
import malaya_speech.train.model.vggvox_v1 as vggvox_v1
import malaya_speech.train.model.vggvox_v2 as vggvox_v2
import malaya_speech.train.model.deep_speaker as deep_speaker
import numpy as np

In [4]:
input_restnet_v1 = tf.placeholder(tf.float32, (None, 512, None, 1))
resnet_v1 = vggvox_v1.model.Resnet1D(
    input_restnet_v1, num_class = 7, mode = 'train'
)
resnet_v1.logits

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.


<tf.Tensor 'dense/BiasAdd:0' shape=(?, 7) dtype=float32>

In [5]:
input_resnet_v2 = tf.placeholder(tf.float32, (None, 257, None, 1))
resnet_v2 = vggvox_v2.model.Resnet1D(
    input_resnet_v2, num_class = 7, mode = 'train'
)
resnet_v2.logits

<tf.Tensor 'prediction/Softmax:0' shape=(?, 7) dtype=float32>

In [6]:
input_deep_speaker = tf.placeholder(tf.float32, (None, None, 64, 1))
model = deep_speaker.model.Model(
    input_deep_speaker, num_class = 7, mode = 'train'
)
model.logits

<tf.Tensor 'dense_1/BiasAdd:0' shape=(?, 7) dtype=float32>

In [7]:
y, sr = malaya_speech.utils.read.wav('../speech/example-speaker/mas-aisyah.wav')
y = malaya_speech.utils.astype.int_to_float(y)
y, sr

(array([ 0.00781274, -0.00222785, -0.00717185, ...,  0.05066073,
         0.0532548 ,  0.05279702], dtype=float32),
 16000)

In [8]:
vggvox_v1_feature = malaya_speech.utils.featurization.vggvox_v1(y)
vggvox_v2_feature = malaya_speech.utils.featurization.vggvox_v2(y)
deep_speaker_feature = malaya_speech.utils.featurization.read_mfcc(y, voice_only = False)

In [9]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [16]:
%%time

sess.run(resnet_v1.logits, feed_dict = {input_restnet_v1:[np.expand_dims(vggvox_v1_feature, -1)]})

CPU times: user 233 ms, sys: 30.3 ms, total: 263 ms
Wall time: 66.7 ms


array([[-0.00066522,  0.01057364,  0.0050069 , -0.06019953, -0.0635055 ,
        -0.00199261,  0.03697133]], dtype=float32)

In [17]:
%%time

sess.run(resnet_v2.logits, feed_dict = {input_resnet_v2:[np.expand_dims(vggvox_v2_feature, -1)]})

CPU times: user 827 ms, sys: 133 ms, total: 960 ms
Wall time: 194 ms


array([[0.145332  , 0.1414403 , 0.14092357, 0.14104506, 0.14368106,
        0.14579006, 0.14178787]], dtype=float32)

In [18]:
%%time

sess.run(model.logits, feed_dict = {input_deep_speaker:[np.expand_dims(deep_speaker_feature, -1)]})

CPU times: user 299 ms, sys: 22.9 ms, total: 322 ms
Wall time: 66.1 ms


array([[ 0.23235865, -0.15339367, -0.42153558,  0.11614576,  0.7868481 ,
         0.04495206, -0.54251367]], dtype=float32)